# Pitch

In [1]:
import penaltyblog as pb
from penaltyblog.matchflow import Flow, get_field, get_x, get_y
import numpy as np
import pandas as pd


In [ ]:
flow = (
    Flow.statsbomb.from_github_file(match_id=3788743)
    .filter(lambda e: get_field("type.name")(e) == "Shot")
    .assign(
        player_name=get_field("player.name"),
        position_name=get_field("position.name"),
        team_name=get_field("team.name"),
        xg=get_field("shot.statsbomb_xg"),
        x=get_x(),
        y=get_y(),        
    )
    .select("minute", "player_name", "position_name", "team_name", "x", "y", "xg")
    .sort("minute")
)

df = flow.to_pandas()

df.head(10)


In [ ]:
flow = (
    Flow.statsbomb.from_github_file(3788743)
    .filter(lambda e: get_field("type.name")(e) == "Shot")
    .assign(
        player_name=get_field("player.name"),
        xg=get_field("shot.statsbomb_xg")
    )
    .group_by("player_name")
    .summarize(
        shots=len,
        total_xg=("xg", "sum"),
        avg_xg_2=("xg", "mean"),
        avg_xg=lambda records: round(
            sum(r["xg"] for r in records if r["xg"] is not None) / len(records), 3
        )
    )
    .sort("total_xg", reverse=True)
    .limit(5)
    .to_pandas()
)

flow

In [4]:
(
    Flow.statsbomb.from_github_file(3788743)
    .assign(
        xg=get_field("shot.statsbomb_xg"),       
    )    
    .summarize(
      total_shots="count",
      mean_xg=("xg","mean"),
      std_xg=("xg","std"),
    )
    .to_pandas()
)

,total_shots,mean_xg,std_xg
0,3858,0.117902,0.142595


In [ ]:
(
    Flow.statsbomb.from_github_file(3788743)
      .filter(lambda e: get_field("type.name")(e) == "Shot")
      .assign(
        player=get_field("player.name"),
        xg=get_field("shot.statsbomb_xg")
      )
      .group_by("player")
      .row_number("xg", new_field="shot_rank", reverse=True)
      .ungroup()
      .select("player", "shot_rank", "xg")
      .to_pandas()
)


In [ ]:
# df = pd.DataFrame({
#     'x': np.random.uniform(0, 100, 11),
#     'y': np.random.uniform(0, 100, 11),
#     'name': [f'Player {i+1}' for i in range(11)]
# })

pitch = pb.pitch.Pitch(width=800, height=400, theme="minimal", provider="statsbomb")
pitch.plot_scatter(df, x='x', y='y', hover='player_name')
pitch.show()


In [ ]:
from penaltyblog.io.statsbomb import load_statsbomb_events
from penaltyblog.pitch import Pitch

# example: Women’s World Cup final (competition 43, season 3, match 8658)
df = load_statsbomb_events(43, 3)

# filter only shots:
shots = df[df['type.name']=='Shot']

pitch = Pitch(provider="statsbomb", theme="classic", title="Shot map—WWC Final")
pitch.plot_scatter(shots, x='x', y='y',
                   hover='player.name', marker=dict(size=12))
pitch.show()


In [ ]:
df2 = pd.DataFrame({
    'x': np.random.uniform(0, 120, 500),
    'y': np.random.uniform(0, 80, 500)
})

# Create the pitch and plot the heatmap
pitch = pb.pitch.Pitch(theme="night")
pitch.plot_heatmap(df2, x='x', y='y')
pitch.show()

In [ ]:
df2 = pd.DataFrame({
    'x': np.random.uniform(10, 50, 500),
    'y': np.random.uniform(20, 75, 500)
})

# Create the pitch and plot the heatmap
pitch = pb.pitch.Pitch(theme="turf")
pitch.plot_kde(df2, x='x', y='y', bandwidth=0.3,)
pitch.show()

In [ ]:
data = pd.DataFrame({
    'x': [30, 50, 70],
    'y': [40, 60, 20],
    'x2': [45, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

# Create a pitch with a dark theme and Opta dimensions
pitch = pb.pitch.Pitch(theme="night")

# Plot the passes as arrows
pitch.plot_arrows(
    data, x='x', y='y', x_end='x2', y_end='y2'
)

# Optional: plot player positions at the start of the arrows
pitch.plot_scatter(data, x='x', y='y', hover='player')

# Show the pitch
pitch.show()


In [ ]:
df = pd.DataFrame({
    'x': [10, 80, 70],
    'y': [10, 60, 20],
    'x2': [40, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

pitch = pb.pitch.Pitch(width=800, height=600, theme="turf", provider="wyscout",)
pitch.plot_comets(df, x='x', y='y', x_end='x2', y_end='y2', segments=20,hover='player')
pitch.show()

In [ ]:
from 